https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

In [ ]:
# prepare data for lstm
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
# https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df =  DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(-i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# load dataset
dataset = read_csv('SeoulBikeData_168.csv', encoding= 'unicode_escape', header=0, index_col=0)

values = dataset.values

In [ ]:
# encode the categorical variables into integers
label_encoder = LabelEncoder()
values[:,12] = label_encoder.fit_transform(values[:,12])
values[:,10] = label_encoder.fit_transform(values[:,10])
values[:,11] = label_encoder.fit_transform(values[:,11])

In [ ]:
# ensure all data is float
values = values.astype('float32')
# normalize features
#scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = scaler.fit_transform(values)

In [ ]:
# frame as supervised learning
hours_to_consider = 168
reframed = series_to_supervised(values, hours_to_consider, 1)

# drop columns we don't want to predict
columns_to_keep = list()
for i in range(337,reframed.shape[1]+1,14):
  columns_to_keep.append(i)

n_cols = [i for i in range (337,reframed.shape[1]+1)]
for i in columns_to_keep:
    n_cols.remove(i)
n_cols2 = [x - 1 for x in n_cols]
print(reframed)
reframed = reframed.drop(reframed.columns[[n_cols2]], axis=1, inplace=False)
print(reframed)

In [ ]:
import pandas as pd
my_list = pd.DataFrame(reframed)
my_list = my_list.columns.values.tolist()

In [ ]:
# split into train and test sets
values = reframed.values
train_ratio = 0.85
n_train_hours = int(365 * hours_to_consider * train_ratio)

train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
train_X, train_y = train[:, 1:], train[:, 0]
test_X, test_y = test[:, 1:], test[:, 0]

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.optimizers import SGD

from keras.metrics import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.metrics import MeanAbsoluteError
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

!pip install keras-tuner
from kerastuner.tuners import RandomSearch, Hyperband
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Reshape

def adj_r2(r2):
  n = train_X.shape[0]
  p = train_X.shape[2]
  return (1-(1-r2)*(n-1)/(n-p-1))

In [ ]:
def model_builder(hp):
  model = Sequential()

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(LSTM(hp_units,
                 input_shape=(train_X.shape[1], train_X.shape[2]),
                 return_sequences=True
                 ))
  model.add(BatchNormalization(axis=-1, 
                            momentum=0.99,
                            epsilon=0.001,
                            center=True,
                            scale=True,
                            beta_initializer='zeros',
                            gamma_initializer='ones',
                            moving_mean_initializer='zeros',
                            moving_variance_initializer='ones',
                            beta_regularizer=None,
                            gamma_regularizer=None,
                            beta_constraint=None,
                            gamma_constraint=None
                            ))

  # model.add(Reshape((train_X.shape[1], train_X.shape[2]), input_shape=(3,)))

  hp_units2 = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(LSTM(hp_units2, input_shape=(train_X.shape[1], train_X.shape[2])))
  model.add(BatchNormalization(axis=-1, 
                            momentum=0.99,
                            epsilon=0.001,
                            center=True,
                            scale=True,
                            beta_initializer='zeros',
                            gamma_initializer='ones',
                            moving_mean_initializer='zeros',
                            moving_variance_initializer='ones',
                            beta_regularizer=None,
                            gamma_regularizer=None,
                            beta_constraint=None,
                            gamma_constraint=None))
  model.add(Dense(1))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('lr', values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
  

  model.compile(optimizer=SGD(lr=hp_learning_rate),
                loss="mse",
                metrics=['accuracy', MeanSquaredError(), RootMeanSquaredError(),MeanAbsoluteError()])

  return model


tuner = Hyperband(model_builder,
                     objective='mean_squared_error',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

stop_early = EarlyStopping(monitor='val_loss', patience=10)

tuner.search(train_X, train_y, epochs=100, validation_split=0.15, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Number of units in the first densely-connected layer: {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('lr')}.
""")

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_X, train_y, epochs=200, validation_split=0.15)

In [ ]:
root_mean_squared_error = history.history['root_mean_squared_error']
best_epoch = val_acc_per_epoch.index(min(root_mean_squared_error)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_X, train_y, epochs=best_epoch, validation_split=0.15)

In [ ]:
# eval_result = hypermodel.evaluate(img_test, label_test)
# print("[test loss, test accuracy]:", eval_result)

In [ ]:
# design network
# model = Sequential()
# model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
# model.add(BatchNormalization(axis=-1, 
#                             momentum=0.99,
#                             epsilon=0.001,
#                             center=True,
#                             scale=True,
#                             beta_initializer='zeros',
#                             gamma_initializer='ones',
#                             moving_mean_initializer='zeros',
#                             moving_variance_initializer='ones',
#                             beta_regularizer=None,
#                             gamma_regularizer=None,
#                             beta_constraint=None,
#                             gamma_constraint=None
#                             ))
# #model.add(Dropout(rate=0.25))
# model.add(Dense(1))




# from sklearn.model_selection import RandomizedSearchCV

# space = {
#     "loss": np.arange(0.1, 1e-6, 0.00001),
#     "validation_split": np.arange(0.1, 0.2, 0.01),
#     "batch_size": [64, 128, 256, 512]
#     }

# search = RandomizedSearchCV(model, space, n_iter=100, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5, random_state=1)
# results = search.fit(train_X, train_y)

# print('Best Score: %s' % result.best_score_)
# print('Best Hyperparameters: %s' % result.best_params_)

# MAX_TRIALS = 20
# EXECUTION_PER_TRIAL = 3

# tuner = RandomSearch(
#     model,
#     objective='mean_squared_error',
#     seed=129537,
#     max_trials=MAX_TRIALS,
#     executions_per_trial=EXECUTION_PER_TRIAL,
#     directory='random_search',
#     project_name='cifar10'
# )





# model.compile(loss='mae',
#               optimizer=SGD(lr=0.001),
#               metrics=["accuracy", 
#                        MeanSquaredError(), 
#                        RootMeanSquaredError(),
#                        MeanAbsoluteError()])

# # fit network
# history = model.fit(train_X, 
#                     train_y,
#                     validation_split=0.15,
#                     epochs=200,
#                     batch_size=60,
#                     verbose=2,
#                     shuffle=False)

# # plot history
# pyplot.plot(history.history['loss'], label='train')
# pyplot.plot(history.history['val_loss'], label='test')
# pyplot.legend()
# pyplot.show()

# # make a prediction
# yhat = model.predict(test_X)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

# # invert scaling for forecast
# oi = np.random.randint(1,101,size=(853,12))
# inv_yhat = concatenate((yhat, oi), axis=1)
# print(inv_yhat.shape)
# inv_yhat = scaler.inverse_transform(inv_yhat)
# inv_yhat = inv_yhat[:,0]

# # invert scaling for actual
# test_y = test_y.reshape((len(test_y), 1))
# inv_y = concatenate((test_y, oi), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
# inv_y = inv_y[:,0]

# # calculate R2
# r2 = r2_score(inv_y, inv_yhat)
# adj_r2 = adj_r2(r2)
# print('Test R2: %.3f' % r2)
# print('Test ADJ R2: %.3f' % adj_r2)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()